In [1]:
import requests
import json
import pandas as pd
import time
import sys
from web3 import Web3

In [2]:
TOKEN_ABI = requests.get('https://raw.githubusercontent.com/pancakeswap/pancake-subgraph/v2/subgraphs/exchange/abis/ERC20.json').json()
FACTORY_ABI = requests.get('https://raw.githubusercontent.com/pancakeswap/pancake-subgraph/v2/subgraphs/exchange/abis/Factory.json').json()
PAIR_ABI = requests.get('https://raw.githubusercontent.com/pancakeswap/pancake-subgraph/v2/subgraphs/exchange/abis/Pair.json').json()

RPC_ENDPOINTS = ['http://176.9.101.209:8545']

WMATIC_ADDRESS = '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270'
USDT_ADDRESS = '0xc2132d05d31c914a87c6611c10748aeb04b58e8f'
USDC_ADDRESS = '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'
WETH_ADDRESS = '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'
MAI_ADDRESS = '0xa3Fa99A148fA48D14Ed51d610c367C61876997F1'
DAI_ADDRESS = '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063'

FACTORY_ADDRESS = Web3.toChecksumAddress('0x5757371414417b8c6caad45baef941abc7d3ab32') #quickswap v2
FACTORY_ADDRESS = Web3.toChecksumAddress('0xc35dadb65012ec5796536bd9864ed8773abc74c4') #sushiswap v2




network = Web3(Web3.HTTPProvider("http://176.9.101.209:8545"))
#network.middleware_onion.inject(geth_poa_middleware, layer=0)


FACTORY_CONTRACT = network.eth.contract(address = FACTORY_ADDRESS, abi = FACTORY_ABI)

In [3]:
class Token():
    def __init__(self, address: str, abi = TOKEN_ABI):
        self.address = Web3.toChecksumAddress(address)
        self.abi = abi
        self.contract = network.eth.contract(address = self.address, abi = self.abi)
        self.symbol = self.contract.functions.symbol().call()
        self.decimals = self.contract.functions.decimals().call()
    def getTokenWMaticPrice(self):
        if self.address == Token(WMATIC_ADDRESS).address:
            return 1.0

        pair = Pair(WMATIC, Token(self.address))
        reserves = pair.getReserves()
        if pair.token0.address == Web3.toChecksumAddress(WMATIC_ADDRESS):
            pair_WMATIC_value, pair_TOKEN_value = reserves
        else:
            pair_TOKEN_value, pair_WMATIC_value = reserves
        return pair_WMATIC_value / pair_TOKEN_value


    @staticmethod
    def getWMATICUSDPrice():
        USD_price = Token(USDC_ADDRESS).getTokenWMaticPrice()
        if USD_price is None:
            return None
        return 1 / USD_price

    def getTokenUSDTPrice(self):
        if self.address == Token(USDT_ADDRESS).address:
            return 1.0

        pair = Pair(USDT, Token(self.address))
        reserves = pair.getReserves()
        if pair.token0.address == Web3.toChecksumAddress(USDT_ADDRESS):
            pair_USDT_value, pair_TOKEN_value = reserves
        else:
            pair_TOKEN_value, pair_USDT_value = reserves
        return pair_USDT_value / pair_TOKEN_value


class Pair():
    def __init__(self, token_A: Token, token_B: Token):
        self.path = [token_A.address, token_B.address]
        self.address = Pair.getAddress(token_A.address, token_B.address)
        if self.address == None:
                raise Exception("Such Pair doesn't exist")
        self.contract = network.eth.contract(address = self.address, abi = PAIR_ABI)
        self.token0 = Token(self.contract.functions.token0().call())
        self.token1 = Token(self.contract.functions.token1().call())

    @staticmethod
    def getAddress(tokenA_address: str, tokenB_address: str):
        factory_contract = network.eth.contract(address = FACTORY_ADDRESS, abi = FACTORY_ABI)
        pair_address = factory_contract.functions.getPair(tokenA_address, tokenB_address).call()
        if pair_address.startswith('0x0000000000000'):
            print("Pair address not found, recheck chosen tokens")
            return None
        return Web3.toChecksumAddress(pair_address)

    def getReserves(self):
        return self.contract.functions.getReserves().call()[:2]

    def getUSDLiquidity(self):
        token0_amount, token1_amount = self.getReserves()
        token0_price = self.token0.getTokenUSDPrice()
        token1_price = self.token1.getTokenUSDPrice()
        return token0_amount * token0_price + token1_amount * token1_price

    def getWBNBLiquidity(self):
        return WBNB.getBalanceOf(self.address)   

In [4]:
WMATIC = Token(WMATIC_ADDRESS)
USDC = Token(USDC_ADDRESS)
USDT = Token(USDT_ADDRESS)

PAIR = Pair(WMATIC,USDT)

In [5]:
PAIR.getReserves()

[20337959461672500992526, 17530066305]

In [16]:
TOKENS = {
    'WMATIC_ADDRESS' : '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
    'USDT_ADDRESS' : '0xc2132d05d31c914a87c6611c10748aeb04b58e8f',
    #'USDC_ADDRESS' : '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174',
    #'WETH_ADDRESS' : '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
    #'MAI_ADDRESS' : '0xa3Fa99A148fA48D14Ed51d610c367C61876997F1',
    'DAI_ADDRESS' : '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063'
    }

EXCHANGES = {
    'QUICKSWAP_2' :'0x5757371414417b8c6caad45baef941abc7d3ab32',
    'SUSHISWAP_2' :'0xc35dadb65012ec5796536bd9864ed8773abc74c4'  
}

In [124]:
 def getPairAddress(tokenA_address: str, tokenB_address: str):
    factory_contract = network.eth.contract(address = FACTORY_ADDRESS, abi = FACTORY_ABI)
    pair_address = factory_contract.functions.getPair(tokenA_address, tokenB_address).call()
    if pair_address.startswith('0x0000000000000'):
        print("Pair address not found, recheck chosen tokens")
        return None
    return Web3.toChecksumAddress(pair_address)
                       


def get_pair_list(token_list,exchanges):
    
    pair_list_address = []
    pair_list = []
    PAIR_RECORD = {}
    
    for EXCHANGE in EXCHANGES.keys():
        FACTORY_ADDRESS = Web3.toChecksumAddress(EXCHANGES[EXCHANGE])
        FACTORY_CONTRACT = network.eth.contract(address = FACTORY_ADDRESS, abi = FACTORY_ABI)
        
        for TOKEN_NAME_0 in TOKENS.keys():
            token0 = Web3.toChecksumAddress(TOKENS[TOKEN_NAME_0])
            for TOKEN_NAME_1 in TOKENS.keys():
                token1 = Web3.toChecksumAddress(TOKENS[TOKEN_NAME_1])
                if not token0 == token1:
                    pair_address = FACTORY_CONTRACT.functions.getPair(token0, token1).call()
                    if not pair_address in pair_list_address:
                        PAIR_RECORD = {}
                        pair_list_address.append(pair_address)
                        PAIR_RECORD['pair_address'] = pair_address 
                        PAIR_RECORD['exchange'] = EXCHANGE
                        
                        pair_list.append(PAIR_RECORD)
                        #df = df.append(PAIR_RECORD, ignore_index=True)    
  #Выше мы собираем список уникальных адресов пар по заданным токенам по выбранным биржам
    
    return pair_list

def build_matrix(pair_list):
    
    df = pd.DataFrame()
    
    for pair in pair_list:
        print (pair['pair_address'])
        RECORD = {}
        PAIR_CONTRACT = network.eth.contract(address = pair['pair_address'], abi = PAIR_ABI)
        reserves = PAIR_CONTRACT.functions.getReserves().call()[:2]
        
        RECORD['pair_address'] = pair['pair_address']
        RECORD['exchange'] = pair['exchange']
        RECORD['reserve_token0']= reserves[0]
        RECORD['reserve_token1']= reserves[1]
        df = df.append(RECORD, ignore_index=True)
    
    return df
    

    
    
    
    
def swap_prediction (reserve0, reserve1, amountin0):
    swap_amount = amountin0
    swap_with_fee = swap_amount*0.997
    amountout1 = reserve1 * swap_with_fee/(reserve0 + swap_with_fee)
    
    print ('BEFORE SWAP')
    print ('Token_0: ',reserve0)    
    print ('Token_1: ',reserve1)
    print ('SWAP DATA')
    print ('Token_0_in: ', swap_amount)
    print ('Token_1_out: ', int(amountout1))
    print ('AFTER SWAP')
    print ('Token_0: ',int(reserve0 + swap_amount))    
    print ('Token_1: ',int(reserve1 - amountout1))


In [126]:
df

,exchange,pair_address,reserves
0,QUICKSWAP_2,0x604229c960e5CACF2aaEAc8Be68Ac07BA9dF81c3,"[833561367093992506128688, 718082302458]"
1,QUICKSWAP_2,0xEEf611894CeaE652979C9D0DaE1dEb597790C6eE,"[12786117809912307891886, 10975717544090093492..."
2,QUICKSWAP_2,0x59153f27eeFE07E5eCE4f9304EBBa1DA6F53CA88,"[92786761740114113054201, 92813489574]"
3,SUSHISWAP_2,0x55FF76BFFC3Cdd9D5FdbBC2ece4528ECcE45047e,"[20368465445070428551199, 17504016326]"
4,SUSHISWAP_2,0x8929D3FEa77398F64448c85015633c2d6472fB29,"[3588404931839703317869, 3077695923278030481344]"
5,SUSHISWAP_2,0x3B31Bb4b6bA4f67F4EF54e78bCb0AAa4f53DC7fF,"[2278367465531011796899, 2279072007]"


In [125]:
df = build_matrix(pair_list)

0x604229c960e5CACF2aaEAc8Be68Ac07BA9dF81c3
0xEEf611894CeaE652979C9D0DaE1dEb597790C6eE
0x59153f27eeFE07E5eCE4f9304EBBa1DA6F53CA88
0x55FF76BFFC3Cdd9D5FdbBC2ece4528ECcE45047e
0x8929D3FEa77398F64448c85015633c2d6472fB29
0x3B31Bb4b6bA4f67F4EF54e78bCb0AAa4f53DC7fF


In [99]:
pair_list = get_pair_list(TOKENS, EXCHANGES)
for i in pair_list:
    print(i)

{'pair_address': '0x604229c960e5CACF2aaEAc8Be68Ac07BA9dF81c3', 'exchange': 'QUICKSWAP_2'}
{'pair_address': '0xEEf611894CeaE652979C9D0DaE1dEb597790C6eE', 'exchange': 'QUICKSWAP_2'}
{'pair_address': '0x59153f27eeFE07E5eCE4f9304EBBa1DA6F53CA88', 'exchange': 'QUICKSWAP_2'}
{'pair_address': '0x55FF76BFFC3Cdd9D5FdbBC2ece4528ECcE45047e', 'exchange': 'SUSHISWAP_2'}
{'pair_address': '0x8929D3FEa77398F64448c85015633c2d6472fB29', 'exchange': 'SUSHISWAP_2'}
{'pair_address': '0x3B31Bb4b6bA4f67F4EF54e78bCb0AAa4f53DC7fF', 'exchange': 'SUSHISWAP_2'}


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [ ]:
unique_pair_list = set(pair_list)